In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

# Read the CSV and Perform Basic Data Cleaning

In [6]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [7]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.tail()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,n,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N
68816,16000.0,0.1131,350.36,MORTGAGE,72000.0,Verified,Jan-2019,low_risk,n,7.02,...,94.3,0.0,1.0,0.0,251486.0,74835.0,23000.0,63090.0,N,N


# Split the Data into Training and Testing

In [8]:
le = LabelEncoder()

In [9]:
le.fit(df["loan_status"])

LabelEncoder()

In [10]:
df["loan_status"] = le.transform(df["loan_status"])

In [11]:
df["loan_status"].head()

0    1
1    1
2    1
3    1
4    1
Name: loan_status, dtype: int32

In [12]:
#for c in df:
#    pd.get_dummies(df, columns=[c])

In [13]:
df = pd.get_dummies(df, columns=["issue_d"
                                 ,"pymnt_plan"
                                 ,"home_ownership"
                                 ,"initial_list_status"
                                 ,"next_pymnt_d"
                                 ,"application_type"
                                 ,"hardship_flag"
                                 ,"debt_settlement_flag"
                                 ,"verification_status"])

In [14]:
#df = df.select_dtypes(exclude=['object'])
#file_path = Path("11-Machine_Learning")
#df.to_csv(file_path, index=False)

In [15]:
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,10500.0,0.1719,375.35,66000.0,1,27.24,0.0,0.0,8.0,0.0,...,1,0,1,1,0,1,1,0,1,0
1,25000.0,0.2000,929.09,105000.0,1,20.23,0.0,0.0,17.0,1.0,...,1,0,1,1,0,1,1,0,0,1
2,20000.0,0.2000,529.88,56000.0,1,24.26,0.0,0.0,8.0,0.0,...,1,0,1,1,0,1,1,0,0,1
3,10000.0,0.1640,353.55,92000.0,1,31.44,0.0,1.0,10.0,1.0,...,1,0,1,1,0,1,1,0,0,1
4,22000.0,0.1474,520.39,52000.0,1,18.76,0.0,1.0,14.0,0.0,...,1,0,1,1,0,1,1,1,0,0


In [16]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]


In [17]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0,0.478007,0.373992,0.148001
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0,0.499520,0.483865,0.355104
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0,0.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000


In [18]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [19]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [20]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [21]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler.fit(X_train,X_test)


StandardScaler()

In [22]:
# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [23]:
rf_model = RandomForestClassifier(random_state=1)

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [25]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
rf_model.fit(X_train_scaled,y_train)
rf_predictions = rf_model.predict(X_test_scaled)

In [26]:
# Calculated the balanced accuracy score
acc_score = accuracy_score(y_test, rf_predictions)
acc_score

0.9960476605637896

In [27]:
# Display the confusion matrix
cm = confusion_matrix(y_test,rf_predictions)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1 "], columns=["Prediction 0", "Prediction 1"])
cm_df

,Prediction 0,Prediction 1
Actual 0,36,65
Actual 1,3,17101


In [28]:
# Print the imbalanced classification report
# YOUR CODE HERE
cr = classification_report(y_test,rf_predictions)
print("Random Forest Classification Report :")
print(cr)

Random Forest Classification Report :
              precision    recall  f1-score   support

           0       0.92      0.36      0.51       101
           1       1.00      1.00      1.00     17104

    accuracy                           1.00     17205
   macro avg       0.96      0.68      0.76     17205
weighted avg       1.00      1.00      1.00     17205



In [29]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08285124628285657, 'total_pymnt_inv'),
 (0.07087614038513551, 'total_rec_prncp'),
 (0.06001682634374264, 'total_rec_int'),
 (0.059967054365370843, 'total_pymnt'),
 (0.050299170590189284, 'last_pymnt_amnt'),
 (0.023273372068498596, 'out_prncp'),
 (0.02225582502037899, 'out_prncp_inv'),
 (0.019860081151011243, 'installment'),
 (0.017851189537141328, 'dti'),
 (0.01598100137159499, 'tot_hi_cred_lim'),
 (0.015859100808807636, 'mo_sin_old_il_acct'),
 (0.015442629946945672, 'mo_sin_old_rev_tl_op'),
 (0.015217079326850059, 'total_il_high_credit_limit'),
 (0.015069282267528, 'max_bal_bc'),
 (0.014983682157896855, 'total_bal_ex_mort'),
 (0.014289616553281452, 'bc_open_to_buy'),
 (0.013743008406085786, 'total_bc_limit'),
 (0.013684383567117587, 'bc_util'),
 (0.013621461447919415, 'loan_amnt'),
 (0.013327700049925364, 'all_util'),
 (0.0133249219131185, 'annual_inc'),
 (0.013313942684007657, 'tot_cur_bal'),
 (0.013049585018685217, 'total_bal_il'),
 (0.012980589959399604, 'total_rev_hi_lim'),
 (

### Easy Ensemble Classifier

In [30]:
EE = EasyEnsembleClassifier()

In [31]:
EE.fit(X_train, y_train)
ee_predictions = EE.predict(X_test)

In [32]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE

In [33]:
ee_balAcc_score = balanced_accuracy_score(y_test, ee_predictions)
ee_acc_score

NameError: name 'ee_acc_score' is not defined

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
ee_cm = confusion_matrix(y_test,ee_predictions)
ee_cm

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
ee_imbal_class = classification_report_imbalanced(y_test,
                                                  ee_predictions)
print(ee_imbal_class)

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE